In [10]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
df = pd.read_csv("https://raw.githubusercontent.com/jtao/AdvancedML/main/data/Auto.csv")
X = df[['cylinders', "displacement", "weight"]]
y = df[["mpg"]]

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Drop non-numeric columns and handle missing values
numeric_df = df.select_dtypes(include=['number']).dropna()

# Calculate correlation coefficients
correlation_matrix = numeric_df.corr()

# Extract correlations with the target variable (mpg)
mpg_correlation = correlation_matrix['mpg'].abs().sort_values(ascending=False)

# Select top 3 features
top_features = mpg_correlation[1:4]  # Exclude the target variable itself
top_features_names = top_features.index.tolist()

print("Top 3 features correlated with mpg:")
print(top_features)

Top 3 features correlated with mpg:
weight          0.831739
displacement    0.804443
cylinders       0.776260
Name: mpg, dtype: float64
